### Analyzing the World Bank's Twitter Feed, Judy Yang, DAT10 Project
### Part 2. Text pre-processing

In [1]:
pwd

u'/Users/judyyang/Documents/GA_DS_course/Final_Project/notebooks'

In [2]:
from datetime import datetime
import time
import json
import pandas as pd
import operator 
import preprocess
from collections import Counter
#from textblob import TextBlob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#% sign 

from ttp import ttp

pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

/Users/judyyang/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
#input the three raw files and append
#WB[0:2]
file1 = pd.read_json('./data/WorldBank_request_01feb_2016a')
#WB[3:31]
file2 = pd.read_json('./data/WorldBank_request_14feb_2016b')
#WB[32:38]
file3 = pd.read_json('./data/WorldBank_request_14feb_2016c')

wb = pd.DataFrame()
wb=wb.append(file1)
wb=wb.append(file2)
wb=wb.append(file3)
wb = wb.reset_index()
wb.shape

(93733, 15)

In [4]:
#Check the list of WB accounts
wb.user_name.value_counts()

World Bank Water        3200
WBG Agriculture         3200
WB Social Enterprise    3200
World Bank Africa       3200
WBG_Finance             3200
WorldBankSouthAsia      3200
World Bank Cities       3200
World Bank Gov          3200
IEG - WB Group          3200
WBGLeadLearnInnovate    3200
World Bank PSD          3200
World Bank Climate      3200
World Bank Health       3200
WBG Environment         3200
World Bank Poverty      3200
World Bank Education    3200
IFC                     3200
WBG Gender              3200
World Bank Data         3200
World Bank Pubs         3200
World Bank Research     3199
World Bank              3196
World Bank Group PPP    2961
World Bank Videos       2725
MIGA                    2000
World Bank India        1907
WB Social Protection    1810
World Bank Kenya        1707
World Bank Energy       1695
World Bank Caribbean    1597
World Bank Transport    1508
World Bank Photos       1259
World Bank Finances     1123
World Bank ECA           808
DIME @ World B

In [5]:
wb.user_name.describe()

count                93733
unique                  38
top       World Bank Water
freq                  3200
Name: user_name, dtype: object

In [6]:
#Check the columns
wb.columns

Index([               u'index',           u'created_at',       u'favorite_count',                   u'id',
              u'retweet_count',               u'source',                 u'text',      u'user_created_at',
           u'user_description', u'user_followers_count',   u'user_friends_count',              u'user_id',
              u'user_location',            u'user_name',     u'user_screen_name'],
      dtype='object')

In [7]:
#Keep select columns for text analysis
keepcols=['id', 'user_screen_name', 'created_at', 'favorite_count', 'retweet_count', 'text']
wb=wb[keepcols]

In [8]:
wb.head()

,id,user_screen_name,created_at,favorite_count,retweet_count,text
0,694299339649347584,WBG_Poverty,2016-02-01 23:20:36,1,1,Eliminating #inequality is not only necessary ...
1,694250314795454467,WBG_Poverty,2016-02-01 20:05:48,0,0,Faster-than-expected #emergingmarket slowdown ...
2,693934929642659840,WBG_Poverty,2016-01-31 23:12:34,0,77,RT @WorldBank: It’s easy to be pessimistic abo...
3,689889192038514688,WBG_Poverty,2016-01-20 19:16:15,0,131,RT @WorldBank: “Data deprivation” makes it dif...
4,643911101600632833,WBG_Poverty,2015-09-15 22:15:44,0,0,More efficient energy sources can provide more...


In [9]:
wb.text.describe()

count                                                 93733
unique                                                90547
top       #Socent create marketplaces to bring skills an...
freq                                                     20
Name: text, dtype: object

In [10]:
#Q: what is unicode exactly?
wb.text.values

array([ u"Eliminating #inequality is not only necessary for realizing one's potential but also for living with dignity https://t.co/1DAIhpnbRr",
       u'Faster-than-expected #emergingmarket slowdown could lower #commodityprices. Will this impact #poverty reduction? https://t.co/IjTj2MXvXK',
       u'RT @WorldBank: It\u2019s easy to be pessimistic about the future. Let your passion drive you. Don\u2019t give up or accept the norm. -Sri Mulyani #Al\u2026',
       ...,
       u'In #Tunisia #IFC support key for #microfinance bank to manage risk during Arab Spring. Story  #A2F13 http://t.co/sKkZeypNm3 @CGAP @IFC_MENA',
       u'#IFC mobile banking products help w/ access to finance for low-income population in #Colombia - #A2F13 http://t.co/njQTKfA1xe - @IFC_LAC',
       u'In #Ethiopia modernized credit registry allows for more financing to entrepreneurs, individuals   #A2F13 http://t.co/njQTKfA1xe @IFCAfrica'], dtype=object)

**Create data vectors**

Make indicator if there is an URL, if there is a RT, if @, and # of words

In [11]:
#text length (should be 0-140 characters)
wb['length'] = wb.text.apply(len)

In [12]:
#Is a RT
sub="RT"
wb['is_RT']=wb.text.str.contains(sub)
wb.is_RT.value_counts()

False    71287
True     22446
Name: is_RT, dtype: int64

In [13]:
# has a hashtag
sub="#"
wb['has_ht']=wb.text.str.contains(sub)
wb.has_ht.value_counts()

True     76759
False    16974
Name: has_ht, dtype: int64

In [14]:
#has a link
sub="http"
wb['has_link']=wb.text.str.contains(sub)
wb.has_link.value_counts()

True     85688
False     8045
Name: has_link, dtype: int64

In [15]:
#has an @
sub="@"
wb['has_at']=wb.text.str.contains(sub)
wb.has_at.value_counts()


False    48793
True     44940
Name: has_at, dtype: int64

**Create lists of text**

https://github.com/edburnett/twitter-text-python

In [16]:
def parse_tags(tweet):
    p=ttp.Parser()   
    results = p.parse(tweet)
    return " ".join(results.tags)

def parse_users(tweet):
    p=ttp.Parser()   
    results = p.parse(tweet)
    return " ".join(results.users)

In [17]:
#create column of hashtags
#create column of links

wb['tags'] = wb.text.apply(parse_tags)
wb['users'] = wb.text.apply(parse_users)
wb.head(2)

,id,user_screen_name,created_at,favorite_count,retweet_count,text,length,is_RT,has_ht,has_link,has_at,tags,users
0,694299339649347584,WBG_Poverty,2016-02-01 23:20:36,1,1,Eliminating #inequality is not only necessary ...,132,False,True,True,False,inequality,
1,694250314795454467,WBG_Poverty,2016-02-01 20:05:48,0,0,Faster-than-expected #emergingmarket slowdown ...,136,False,True,True,False,emergingmarket commodityprices poverty,


In [18]:
#Q: how to create all the tags together, some are one tag, some are 2 tags
wb.tags.value_counts(ascending=False).head()

           17128
PPPs         786
Africa       714
ecourse      675
goodgov      624
Name: tags, dtype: int64

In [19]:
wb.users.value_counts(ascending=False).head()

                   49197
WorldBank           5084
JimKim_WBG          1026
IFC_org              545
WorldBankAfrica      409
Name: users, dtype: int64

In [20]:
#use to count all tags and users
#note, only works when it is in a list, not string
#http://stackoverflow.com/questions/32851683/pandas-convert-list-to-string-wide-to-long-with-one-column
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

'''
# since you have a column of lists skip any type of string munging
# instead, go straight to converting the lists to a Series
s = df['ID'].apply(Series,1).stack()
s.index = s.index.droplevel(-1)
s.name = 'ID'
del df['ID']
df = df.join(s)

# since you have a column of lists skip any type of string munging
# instead, go straight to converting the lists to a Series
s = wb['users'].apply(Series,1).stack
#s.index = s.index.droplevel(-1)
#s.name = 'users'
#del wb['users']
#test = wb.join(s)
#test
s
'''

"\n# since you have a column of lists skip any type of string munging\n# instead, go straight to converting the lists to a Series\ns = df['ID'].apply(Series,1).stack()\ns.index = s.index.droplevel(-1)\ns.name = 'ID'\ndel df['ID']\ndf = df.join(s)\n\n# since you have a column of lists skip any type of string munging\n# instead, go straight to converting the lists to a Series\ns = wb['users'].apply(Series,1).stack\n#s.index = s.index.droplevel(-1)\n#s.name = 'users'\n#del wb['users']\n#test = wb.join(s)\n#test\ns\n"

**Text clean up**

http://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/

1) Escape HTML characters
2) Decode data
3) Apostrophe lookup
4) Remove stop words
5) Remove punctuation
6) Remove expressions
7) Split attached words
8) Slang lookup
9) Standardizing words
10) Remove URLs

In [21]:
#create clean text for text analysis: remove URL, remove #, remove @, just words
import HTMLParser
import re
html_parser = HTMLParser.HTMLParser()
wb['text_clean'] = html_parser.unescape(wb.text)


In [22]:
#10. remove htmls
wb['text_clean'] = wb.text.str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ")

In [23]:
#Remove numbers

#Remove word amp



In [24]:
cols =['text_clean']
print wb[cols]

                                              text_clean
0      Eliminating  inequality is not only necessary ...
1      Faster than expected  emergingmarket slowdown ...
2      RT    It s easy to be pessimistic about the fu...
3      RT     Data deprivation  makes it difficult to...
4      More efficient energy sources can provide more...
5      Women improve  financialinclusion of the poore...
6      RT    We want to reduce  violence in  Jamaica ...
7      Rapid economic growth between 2002  amp  2008 ...
8      Laws in close to 155 economies make it difficu...
9      Equity  amp  learning will help  endextremepov...
10     In 100 countries  women are barred from some j...
11     In  India  women run groups help local village...
12     A World Bank financed project improves women s...
13     New Global Financial Development Report out  F...
14     How do children of the bottom 40  fare against...
15     How can laws help women achieve their full eco...
16     How does climate change 

** Create some more variables **

In [25]:
# 1) Create dependent variables
wb['retweet_high']= np.where(wb.retweet_count>1, 1, 0)
wb['favorite_high']= np.where(wb.favorite_count>1, 1, 0)

In [ ]:
# 2) Create dummy variables for each account
Iuser = pd.get_dummies(wb.user_screen_name, prefix='Iuser')
# concatenate the original DataFrame and the dummy DataFrame
wb = pd.concat([wb, Iuser], axis=1)

In [ ]:
# 3) Create time columns and dummies, and append
wb['year']=wb.created_at.apply(lambda x: x.strftime('%y'))
wb['year2'] = wb.year.astype('category', categories=["09", "10", "11" ,"12", "13", "14" , "15", "16"], ordered=True)

wb['year_month']=wb.created_at.apply(lambda x: x.strftime('%y-%m'))
wb['hour']=wb.created_at.apply(lambda x: x.strftime('%H'))
#wb['hour_cat'] = wb.hour.astype('category', categories=["00", "01", "02", "03", "04", "05", "06", "07","08" ,"09", "10", "11" ,"12", "13", "14" , "15", "16", "17", "18", "19", "20", "21", "22", "23"], ordered=True)
#Ihour= pd.get_dummies(wb.hour, prefix='Ihour')
#wb = pd.concat([wb, Ihour], axis=1)

## Save cleaned data

In [ ]:
wb.to_json('./data/WorldBank_all_processed_14feb_2016')